In [3]:
from IPython.display import clear_output
!pip install torch
!pip install torchvision
!pip install opencv-python
!pip install seaborn
!pip install matplotlib
!pip install cleverhans
!pip install cachetools
!pip install pandas
!apt-get update && apt-get install ffmpeg libsm6 libxext6  -y
!pip install captum
!pip install cleverhans
clear_output()

In [4]:
import numpy as np
from scipy.stats import entropy
from scipy import ndimage
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
from torch.utils.data import DataLoader
import torchvision.transforms as transforms
l1_dist = lambda x1,x2: np.sum(np.abs(x1 - x2), axis=tuple(range(len(x1.shape))[1:]))
l2_dist = lambda x1,x2: np.sum((x1-x2)**2, axis=tuple(range(len(x1.shape))[1:]))**.5
import featuresqueeze
from load_skin import load_skin_data
from load_diabetic import load_diabetic_data

/opt/conda/lib/python3.10/site-packages/torch/cuda/__init__.py:138: UserWarning: CUDA initialization: The NVIDIA driver on your system is too old (found version 11040). Please update your GPU driver by downloading and installing a new version from the URL: http://www.nvidia.com/Download/index.aspx Alternatively, go to: https://pytorch.org to install a PyTorch version that has been compiled with your version of the CUDA driver. (Triggered internally at ../c10/cuda/CUDAFunctions.cpp:108.)
  return torch._C._cuda_getDeviceCount() > 0


In [5]:
# Device configuration
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [6]:
def load_model(path):
    model = torch.load(path)
    model.to(device)
    model.train(False)
    return model

## Skin 

In [7]:
path = "model/skin/model_skin.pth"
model = load_model(path)
model.to(device)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [8]:
train_loader, val_loader = load_skin_data()

/root/load_skin.py:107: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_train=df_train.append([df_train.loc[df_train['cell_type_idx'] == i,:]]*(data_aug_rate[i]-1), ignore_index=True)


In [9]:
FPR = [0.1]
for fpr in FPR:
    t=[]
    for step, (x_batch, y_batch) in enumerate(train_loader):
        x_batch, y_batch = x_batch.to(device), y_batch.to(device)
        threshold = featuresqueeze.train_fs(model, x_batch, fpr)
        t.append(threshold)
        if step==100:
            break
    print("Threshold for {} FPR is {}.".format(fpr, sum(t)/len(t)))

Threshold for 0.1 FPR is 1.8337416991148845.


In [15]:
from cleverhans.torch.attacks.projected_gradient_descent import projected_gradient_descent
from cleverhans.torch.attacks.fast_gradient_method import fast_gradient_method

threshold = 1.8337
eps = [8/255, 16/255, 32/255, 64/255]

def make_fgsm_attack(x_batch, y_batch, eps, normal_model): 
    images_pgd = fast_gradient_method(normal_model, x_batch, eps, np.inf)
    _, y_pred_pgd = normal_model(images_pgd).max(1)
    return images_pgd, y_pred_pgd

def make_pgd_attack(x_batch, y_batch, eps, normal_model): 
    alpha = eps/10
    steps = int(alpha*eps)
    images_pgd = projected_gradient_descent(normal_model, x_batch, eps, alpha, steps, np.inf)
    _, y_pred_pgd = normal_model(images_pgd).max(1)
    return images_pgd, y_pred_pgd

def pgd_adv_tpr(model, val_loader):
    rest = []
    resf = []
    for ep in eps:
        t=[]
        f=[]
        for step, (x_batch, y_batch) in enumerate(val_loader):
            x_batch, y_batch = x_batch.to(device), y_batch.to(device)
            images_adv,y_pred_adv = make_pgd_attack(x_batch, y_batch, ep, model)
            images_adv, y_pred_adv = images_adv.to(device), y_pred_adv.to(device)
            tpr, fpr, tp, ap, fp, an = featuresqueeze.evaluate_test(model, images_adv, threshold)
            t.append(tpr)
            f.append(fpr)
            if step==100:
                break
        avg = sum(t)/len(t)
        rest.append(avg)
        avg = sum(f)/len(f)
        resf.append(avg)
    return rest, resf

def fgsm_adv_tpr(model, val_loader):
    rest = []
    resf = []
    for ep in eps:
        t=[]
        f=[]
        for step, (x_batch, y_batch) in enumerate(val_loader):
            x_batch, y_batch = x_batch.to(device), y_batch.to(device)
            images_adv,y_pred_adv = make_fgsm_attack(x_batch, y_batch, ep, model)
            images_adv, y_pred_adv = images_adv.to(device), y_pred_adv.to(device)
            tpr, fpr, tp, ap, fp, an = featuresqueeze.evaluate_test(model, images_adv, threshold)
            t.append(tpr)
            f.append(fpr)
            if step==100:
                break
        avg = sum(t)/len(t)
        rest.append(avg)
        avg = sum(f)/len(f)
        resf.append(avg)
    return rest, resf


In [ ]:
print("PGD")
print(pgd_adv_tpr(model, val_loader))

PGD
([0.028079710144927536, 0.04076086956521739, 0.05434782608695652, 0.019021739130434784], [nan, nan, nan, nan])


In [17]:
print("FGSM")
print(fgsm_adv_tpr(model, val_loader))

FGSM
([0.3309782608695652, 0.35265700483091783, 0.12246376811594202, 0.002717391304347826], [nan, nan, nan, nan])


## Diabetic

In [18]:
path = "model/diabetic/model_diabetic.pth"
model = load_model(path)
model.to(device)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [19]:
train_loader, val_loader = load_diabetic_data()

In [20]:
FPR = [0.1]
for fpr in FPR:
    t=[]
    for step, (x_batch, y_batch) in enumerate(train_loader):
        x_batch, y_batch = x_batch.to(device), y_batch.to(device)
        threshold = featuresqueeze.train_fs(model, x_batch, fpr)
        t.append(threshold)
        if step==100:
            break
    print("Threshold for {} FPR is {}.".format(fpr, sum(t)/len(t)))

Threshold for 0.1 FPR is 1.1580363736294284.


In [21]:
threshold = 1.158

In [22]:
print("PGD")
print(pgd_adv_tpr(model, val_loader))

PGD
([0.10112577639751553, 0.09976708074534162, 0.12247670807453416, 0.23544254658385091], [nan, nan, nan, nan])


In [ ]:
print("FGSM")
print(fgsm_adv_tpr(model, val_loader))

FGSM
([0.3526785714285714, 0.3579192546583851, 0.5351319875776397, 0.5065993788819876], [nan, nan, nan, nan])
